# 📘 Pic2Plate: Gemini Recipe Generator and Nutrition Analyzer
This notebook generates a recipe based on user-provided ingredients using Gemini and calculates its nutritional values.

It demonstrates how to:
  1. Generate a recipe with a structured output from a list of ingredients using Gemini.
  2. Extract structured ingredient data from the recipe using few-shot prompting. 
  3. Calculate the nutritional value of the generated recipe by function calling and querying a USDA food database.
  4. Detect food items and quantities from an image, generate a recipe based on them, and calculate its calories and nutritional content.

## 🔧 Setup and Imports

In [2]:
# Install the Gemini GenAI Python SDK
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.7 MB/s eta 0:00:00


In [21]:
# Load API key securely from Kaggle secrets and set it as environment variable
import os
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# Import necessary standard libraries
from google import genai
from google.genai import types
from collections import defaultdict
import json

client = genai.Client(api_key=GOOGLE_API_KEY)
MODEL_ID = "gemini-2.0-flash"

# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

## 🥕 Input Ingredients


In [22]:
# Example usage 
ingredients = ["Tomatoes", "beef", "carrots"]
prep_time = "45 minutes"
cuisine_style = "Mexican"
dietary_restrictions = [""]

### 🍳 Generate Recipe from Ingredients
Use Gemini to generate a detailed recipe based on the provided ingredients.

The response is a structured JSON output (defined by `Recipe` schema) and includes recipe title, ingredient list, instructions, prep time, cuisine style and dietary restrictions.

In [24]:
# Define the expected recipe schema and a function to generate recipes using Gemini
### Define output schema
import typing_extensions as typing

class Recipe(typing.TypedDict):
    title: str
    ingredients: list[str]
    instructions: list[str]
    prep_time: str
    cusine_style: str
    dietary_restrictions: list[str]
    type: str

def generate_recipe_gemini(ingredients, prep_time, cuisine_style, dietary_restrictions=None):
    """
    Generates a recipe using Gemini based on user input.

    Args:
        ingredients (list): List of ingredients provided by the user.
        prep_time (str): Desired preparation time (e.g., "30 minutes", "1 hour").
        cuisine_style (str): Desired cuisine style (e.g., "Italian", "Mexican", "Indian").
        dietary_restrictions (list, optional): List of dietary restrictions (e.g., "vegetarian", "gluten-free").

    Returns:
        dict: A JSON dictionary containing the recipe title, ingredients, instructions, and other relevant information, or an error message.
    """

    prompt = f"Generate a recipe for a {cuisine_style} dish. "
    if ingredients:
        prompt += f"Use these ingredients: {', '.join(ingredients)}. "
    if prep_time:
        prompt += f"I have about {prep_time} to prepare it. "
    if dietary_restrictions:
        prompt += f"The recipe should be {', '.join(dietary_restrictions)}. "

    prompt += "Please provide the recipe title, ingredients list, and step-by-step instructions. "
    
    response = client.models.generate_content(
                model=MODEL_ID,
                contents=prompt,
                config={
        'response_mime_type': 'application/json',
        'response_schema': Recipe,
    },
    )
    return response.text      

recipe = generate_recipe_gemini(ingredients, prep_time, cuisine_style, dietary_restrictions)
print(recipe)

{
  "title": "Speedy Mexican Beef Skillet",
  "ingredients": [
    "1 pound ground beef",
    "1 tablespoon olive oil",
    "1 medium onion, chopped",
    "2 carrots, peeled and diced",
    "2 cloves garlic, minced",
    "1 (14.5 ounce) can diced tomatoes, undrained",
    "1 (15 ounce) can black beans, rinsed and drained",
    "1 cup beef broth",
    "1 tablespoon chili powder",
    "1 teaspoon cumin",
    "1/2 teaspoon smoked paprika",
    "Salt and pepper to taste",
    "Optional toppings: shredded cheese, sour cream, cilantro, avocado"
  ],
  "instructions": [
    "Heat olive oil in a large skillet over medium-high heat. Add ground beef and cook, breaking it up with a spoon, until browned. Drain off any excess grease.",
    "Add chopped onion and diced carrots to the skillet and cook until softened, about 5 minutes.",
    "Stir in minced garlic, diced tomatoes (undrained), black beans, beef broth, chili powder, cumin, smoked paprika, salt, and pepper.",
    "Bring the mixture to a s

## 🥗 Calculate the nutrition and calorie of the generated recipe
The calculation process involves three steps and two self-defined functions: : `extract_ingredients` and `calculate_nutrition`. 

- Step 1: Extract ingredient names and quantities from the generated recipe text. This step uses Gemini’s few-shot prompting technique and saves the output in a structured JSON format. → Function `extract_ingredients`
- Step 2: For each extracted ingredient, retrieve the nutrition information by matching it against the USDA Foundation Food database. This process follows a two-step matching logic: first, match by generic name; second, refine the match using a specific name, if available. → Function `calculate_nutrition`
- Step 3: For each matched ingredient, retrieve its nutritional profile and adjust by the corresponding quantity. The total nutritional values of the recipe are obtained by summing the values of all ingredients. → Function `calculate_nutrition`

The entire process is orchestrated through the function-calling feature of Gemini. The function that triggers the above steps is
`get_recipe_nutrition`. 

### Step 1: Extract ingredient names and their quantities

In [9]:
# Function to extract normalized ingredient names and quantities from recipe text using Gemini
def extract_ingredients(recipe_text: str) -> dict:
    """
    Use GenAi to extract ingredients from a recipe text generated by LLM.
    
    Args:
        recipe_text (str): Raw recipe text containing ingredients and their quantities.
    
    Returns:
        dict: A JSON dictionary with ingredient names (as keys) and their quantities (as values).
    """

    prompt = f"""
    Extract the ingredients and their quantities from the following recipe. \ 
              Return the result as a JSON dictionary where keys are ingredient names (singular) and values are integers.
              The ingredient names should be normalized to standardize variations (e.g., 'bell pepper' → 'pepper, bell', 'olive oil' -> 'oil, olive').
    Ingredients:
    {recipe_text}
    """

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config={
            'response_mime_type': 'application/json'}
    )

    ingredient_counts = eval(response.text)
    return eval(response.text)

### Step 2&3: Nutrition information retrieving and total nutrition calculation

In [11]:
# Calculate total nutritional values by matching ingredients with USDA database and scaling by quantity
def calculate_nutrition(ingredient_counts):
    with open('/kaggle/input/usda-foundationfood/USDAfoundationFood.json', 'r') as file:   
        usda_data = json.load(file) ## [Improvement to do: Move this logic outside the loop to allow users the option to choose which database to query.]
    # get the list of foods in the dataset
    foods = usda_data["FoundationFoods"]

    ## Define the important nutrients
    important_nutrients = [
    "Energy", "Protein", "Total lipid (fat)", "Carbohydrate, by difference",
    "Fiber, total dietary", "Sugars, total including NLEA",
    "Sodium, Na", "Potassium, K", "Vitamin C, total ascorbic acid",
    "Iron, Fe", "Calcium, Ca"
]

    totals = defaultdict(float)

    for food_name, count in ingredient_counts.items():
        count = int(count)
        # Step 1: Find the corresponding food in the USDA database
        ## Split the food name into two parts -- generic name and specific name, following the USDA format
        parts = food_name.split(',')
        generic_name = parts[0].strip()  # First part (e.g., "oil")
        specific_name = parts[1].strip() if len(parts) > 1 else ''  # Second part (e.g., "olive")
    
        # Debugging: Check the parts
        #print(f"Generic Name: '{generic_name}', Specific Name: '{specific_name}'")
    
        # Step 2: Find matches based on generic name
        matches = [f for f in foods if generic_name.lower() in f["description"].lower()]

        if not matches:    # Ignore the food if not found in the USDA database
            #print(f"No match found for '{food_name}'")
            continue

        if specific_name:  # If generic name matches, 
            # loop through all matches to find one with the specific name
            matched_food = None
            for food in matches:
                if specific_name.lower() in food["description"].lower():
                    matched_food = food
                    #print(f"Using '{matched_food['description']}' for '{food_name}'")
                    break  # Exit the loop once a match is found

            if matched_food == None: 
               matched_food = matches[0]  # if couldn't find a match for the specific name, use the first match of generic name. 
               #print(f"Using '{matched_food['description']}' for '{food_name}'")

        else:
            # If no specific name match, just use the first match
            matched_food = matches[0]
            #print(f"Using '{matches[0]['description']}' for '{food_name}'")
        

        # Step 2: Get gram weight of one unit (from foodPortions)
        try:
            portion = matched_food["foodPortions"][0]
            grams_per_item = portion["gramWeight"]
        except (KeyError, IndexError):
            #print(f"No portion size found for '{food_name}', assuming 100g")
            grams_per_item = 100  # fallback assumption

        total_grams = grams_per_item * count

        # Step 3: Add scaled nutrients
        for nutrient in matched_food["foodNutrients"]:
            name = nutrient["nutrient"]["name"]
            unit = nutrient["nutrient"].get("unitName", "")
            amount_per_100g = nutrient.get("amount")
            if name in important_nutrients and amount_per_100g is not None:
                scaled_amount = amount_per_100g * total_grams / 100  # scale to actual weight
                totals[(name, unit)] += scaled_amount

    return totals

### Declare a function `get_recipe_nutrition` to call the above two functions and to be used by LLM
This function executes the two self-defined functions above:

- `recipe_ingredients = extract_ingredients(recipe_text)`
- `nutrition = calculate_nutrition(recipe_ingredients)`

In [35]:
# Define function declaration for LLM function calling and implement the function logic
## This is the function that LLM can call to extract the ingredients and then calculate nutrition and calorie
get_recipe_nutrition_declaration = {
    "name": "get_recipe_nutrition",
    "description": "Calculate the nutrition value of a given recipe.",
    "parameters": {
        "type": "object",
        "properties": {
            "recipe_text": {
                "type": "string",
                "description": "Recipe text generated by LLM",
            },
        },
        "required": ["recipe_text"],
    },
}

# This is the actual function that would be called based on the model's suggestion
def get_recipe_nutrition(recipe_text: str) -> dict:
    """
    Retrieve and calculate the nutrition of a given recipe by extracting ingredients 
    and querying a food database.
    
    Args:
        recipe_text (str): Raw recipe text containing ingredients and their quantities.    
    Returns:
        dict: A dictionary with the total nutritional values for the recipe (e.g., calories, protein, fat).
    """
    # Step 1: Extract ingredients from recipe
    recipe_ingredients = extract_ingredients(recipe_text)
    # Step 2&3: Calculate the nutritional information
    nutrition = calculate_nutrition(recipe_ingredients)
    
    return nutrition


### Finally, get the calculated nutrition and calorie.
This is done by calling LLM with function declarations and executing the function code.

In [40]:
## Generation Config with Function Declaration
tools = types.Tool(function_declarations=[get_recipe_nutrition_declaration])
config = types.GenerateContentConfig(
    temperature=0,
    tools=[tools]
)

# pass the recipe text to LLM via contents. Note that the recipe is not passed by LLM to the functions in its entirety. 
# Instead, the LLM selects the information it deems relevant based on its analysis.
# This can be modified to pass the recipe text as it is. This is a to-do. 
contents=[types.Content(role="user", parts=[types.Part(text=recipe)])]  

## call LLM
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=contents,
    config=config
)

## execute the function code
tool_call = response.candidates[0].content.parts[0].function_call

if tool_call.name == "get_recipe_nutrition":
    result = get_recipe_nutrition(**tool_call.args)
    print(f"Function execution result: {result}")

Function execution result: defaultdict(<class 'float'>, {('Iron, Fe', 'mg'): 9.956779999999998, ('Potassium, K', 'mg'): 2753.37, ('Sodium, Na', 'mg'): 575.398, ('Total lipid (fat)', 'g'): 20.083800000000004, ('Calcium, Ca', 'mg'): 343.296, ('Protein', 'g'): 60.431039999999996, ('Carbohydrate, by difference', 'g'): 87.7066, ('Energy', 'kcal'): 462.276, ('Vitamin C, total ascorbic acid', 'mg'): 24.7232, ('Energy', 'kJ'): 1930.6200000000001, ('Fiber, total dietary', 'g'): 16.4848})


## 🖼️➡️🥗 Image-to-Recipe Experiment: From Photo to Nutrition
What we're testing:
Upload an image to Gemini, have it identify the ingredients, generate a recipe, and provide a nutritional breakdown based on the contents of the image.

Step 1: Detect food items in the image and generate a recipe.
Gemini analyzes the image, identifies the food items, and produces a recipe using the detected ingredients.

Step 2: Calculate the nutrition from the image-based recipe.
Currently, this is generated by Gemini. My to-do is to replace this with my own logic using the `extract_ingredients` and `calculate_nutrition` functions.


In [48]:
from PIL import Image
from google import genai

image = Image.open("/kaggle/input/vegetable-test/vegetables.png")
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[image, "tell me what food and how many in this image. Then generate a recipe use food in this image and tell me the nutrition of that recipe "],
)
print(response.text)

Here's an analysis of the image and a recipe:

**Food Identification and Count:**

Based on the image, here's a breakdown of the visible foods:

*   **Tomatoes:** 4
*   **Broccoli:** 2
*   **Cauliflower:** 1
*   **Carrots:** 3
*   **Red Cabbage:** 1
*   **Bell Peppers:** 3 (red, yellow, orange)
*   **Zucchini:** 2
*   **Onions:** 2

**Recipe: Roasted Vegetable Medley**

This recipe utilizes many of the vegetables in the image.

*Ingredients:*

*   1 cup Broccoli florets
*   1 cup Cauliflower florets
*   2 medium Tomatoes, quartered
*   1 medium Carrot, sliced
*   1/2 medium Red Cabbage, chopped
*   1/2 each of Red, Yellow, and Orange Bell Peppers, seeded and chopped
*   1 medium Zucchini, sliced
*   1/2 medium Onion, chopped
*   2 tablespoons Olive Oil
*   1 teaspoon Italian Seasoning
*   Salt and Pepper to taste

*Instructions:*

1.  Preheat oven to 400°F (200°C).
2.  In a large bowl, combine all the vegetables.
3.  Drizzle with olive oil and sprinkle with Italian seasoning, salt, and